# IFC Rasterization for Efficient Spatial Analysis

## Intro

### Import libraries and the IFC model:

In [1]:
import pyvista as pv
import numpy as np
from typing import Any, Tuple
import ifcopenshell
from importlib import reload 
import multiprocessing
import ifcopenshell
import ifcopenshell.geom
import time
import vtk
ifc_file = ifcopenshell.open(r"IFC Files\Project1.ifc")
#ifc_file = ifcopenshell.open(r"IFC Files\Clinic.ifc")
#ifc_file = ifcopenshell.open(r"IFC Files\Duplex.ifc")

## Functions

In [2]:


def vtk_block_by_building_element(ifc_file):
    building_elements = ifc_file.by_type("IfcBuildingElement")
    settings = ifcopenshell.geom.settings()
    settings.set(settings.USE_WORLD_COORDS, True)
    settings.set(settings.APPLY_DEFAULT_MATERIALS, True)
    iterator = ifcopenshell.geom.iterator(settings, ifc_file, multiprocessing.cpu_count())
    multiblock = pv.MultiBlock()
    multiblock_openings = pv.MultiBlock()

    element_information = {} # Dictionary to hold element information
    exclude_list = ["IfcSpace", "IfcOpeningElement"]
    
    if iterator.initialize():
        while True:
            shape = iterator.get()
            if shape.type not in exclude_list:
                element = ifc_file.by_guid(shape.guid)
                        
                faces = shape.geometry.faces
                verts = shape.geometry.verts
                poly_data = pv.PolyData(list(verts), to_vtk_faces(faces))
                multiblock.append(poly_data)
                  
                if element in building_elements:
                #print(element.all_attributes()) --> why doesn't it work?
                        
                    element_information[shape.guid] = {
                    "Geo": poly_data, 
                    "Type": shape.type,
                    "Name": element.Name,
                    "Description": element.Description
                    }


            if not iterator.next():
                break
                
    return multiblock, element_information

def create_uniform_grid(bounds, voxel_size):
    """Create a uniform grid within the given bounds."""
    x = np.arange(bounds[0], bounds[1] + voxel_size, voxel_size)
    y = np.arange(bounds[2], bounds[3] + voxel_size, voxel_size)
    z = np.arange(bounds[4], bounds[5] + voxel_size, voxel_size)
    return pv.StructuredGrid(*np.meshgrid(x, y, z))


def boxes_touch(A, B):
    """
    Check if two 3D bounding boxes touch or overlap.
    
    Parameters:
    - A, B: Tuples representing the bounds of boxes A and B.
      Each tuple should be in the format (xmin, xmax, ymin, ymax, zmin, zmax).
    
    Returns:
    - True if the boxes touch or overlap, otherwise False.
    """
    
    # Check for overlap in the x, y, and z dimensions
    overlap_x = A[0] <= B[1] and A[1] >= B[0]
    overlap_y = A[2] <= B[3] and A[3] >= B[2]
    overlap_z = A[4] <= B[5] and A[5] >= B[4]
    
    # Return True if all dimensions overlap, otherwise False
    return overlap_x and overlap_y and overlap_z

def voxelize_space(meshes, voxel_size, mesh_info):

    """Voxelize space and check intersections with given mesh."""
    opening_list = ["IfcDoor", "IfcWindow"]

    grid = create_uniform_grid(meshes.bounds, voxel_size)

    num_points = grid.cell_centers().n_points

    mask = np.zeros(num_points, dtype=bool)

    cell_to_guid = {}

    print(f'Total number of voxels: {num_points}')

    select_enclosed = vtk.vtkSelectEnclosedPoints()
    
    select_enclosed.SetInputData(grid.cell_centers())

    for guid, mesh_data in mesh_info.items():
        mesh = mesh_data['Geo']
        mesh_type = mesh_data['Type']
        mesh_bounds = mesh.bounds
        
        select_enclosed.SetSurfaceData(mesh)

        select_enclosed.Update()

        for i in range(num_points):
            if mesh_type in opening_list:
                cell = grid.extract_cells(i)
                cell_bounds = cell.bounds   

                if boxes_touch(mesh_bounds, cell_bounds):
                    mask[i] = True
                    cell_to_guid[i] = guid
            else:    
                if not mask[i] and select_enclosed.IsInside(i):
                    
                    mask[i] = True
                    cell_to_guid[i] = guid
                    
    return grid, mask, cell_to_guid

## Voxelization

In [3]:
meshes_all, info = vtk_block_by_building_element(ifc_file)

p = pv.Plotter()

start_time = time.time()

voxel_size = 0.2

# Voxelize the entire space of the combined mesh
grid, mask, cell_info = voxelize_space(meshes_all, voxel_size, info)


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken to voxelize: {elapsed_time:.4f} seconds")

# Visualization
p.add_mesh(grid, opacity=0.3, show_edges=True)

p.add_mesh(grid.extract_cells(np.where(mask)[0]), color="red", opacity=0.5)

p.show()

Total number of voxels: 10944
Time taken to voxelize: 20.9682 seconds


Widget(value="<iframe src='http://localhost:50566/index.html?ui=P_0x2ba7d242b00_0&reconnect=auto' style='width…